In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import ntpath
import csv
import datetime
import fnmatch
import os
import math
import arcpy

In [56]:
df = pd.read_excel(r"Z:\NSFSCCTrajectories\KeystoneAndMcCarran\New folder\conflictSum_Integration_1.xlsx")
#df_out = pd.concat(listed_df)
df_out  = df

In [3]:
!pip install xlsxwriter

Defaulting to user installation because normal site-packages is not writeable


In [57]:
df_out['timeDiff'] = df_out['timeDiff'].abs() / 10


lon1=numpy.radians(df_out["Longitude"])
lat1=numpy.radians(df_out["Latitude"])
lon2=numpy.radians(df_out["Longitude_cf"])
lat2=numpy.radians(df_out["Latitude_cf"])
R = 6371000
    #radius of the Earth meters, 6371 kim
    #change in coordinates
dlon = lon2 - lon1
dlat = lat2 - lat1
    #Haversine formula
a = numpy.sin(dlat/2)**2 + numpy.cos(lat1) * numpy.cos(lat2) * numpy.sin(dlon / 2)**2
c = 2 * numpy.arctan2(numpy.sqrt(a), numpy.sqrt(1 - a))
df_out['Distance'] = R*c* 3.28084


df_out['ObjectIDs'] = list(zip(df_out.ObjectID, df_out.ObjectID_cf))
df_out['ObjectIDs'] = [tuple(sorted(x)) for x in df_out['ObjectIDs']]

df_group = df_out.groupby('ObjectIDs',as_index=False)['Distance'].min()
df_merge = pd.merge(df_group,df_out,on = ['ObjectIDs','Distance'],how='left')
df_out = df_merge.drop_duplicates(['ObjectIDs'])

#df_group = df_out.groupby('ObjectIDs',as_index=False)['timeDiff'].min()
#df_merge = pd.merge(df_group,df_out,on = ['ObjectIDs','timeDiff'],how='left')
#df_out = df_merge.drop_duplicates(['ObjectIDs'])

#df.sort_values(['ObjectIDs','Distance'],ascending=True)
#df_out.drop_duplicates(subset = 'ObjectIDs',keep='first',inplace=True)

#df_out.sort_values('Distance', ascending=True).drop_duplicates('ObjectIDs').sort_index()

df_out['DateTime'] = pd.to_datetime(df_out['DateTime'])
df_out['DateTime_cf'] = pd.to_datetime(df_out['DateTime_cf'])

df_out['Height'] = df_out['Height']*3.28084
df_out['Height_cf'] = df_out['Height_cf']*3.28084
df_out['Width'] = df_out['Width']*3.28084
df_out['Width_cf'] = df_out['Width_cf']*3.28084
df_out['Length'] = df_out['Length']*3.28084
df_out['Length_cf'] = df_out['Length_cf']*3.28084
df_out['Max_Length'] = df_out['Max_Length']*3.28084
df_out['Max_Length_cf'] = df_out['Max_Length_cf']*3.28084
df_out['Area'] = df_out['Area']*10.764
df_out['Area_cf'] = df_out['Area_cf']*10.764

'''
conditions = [(df_out['Class'] == 1) & (df_out['Class_cf'] == 1),
            (df_out['Class'] == 2) & (df_out['Class_cf'] == 2),
            (df_out['Class'] == 3) & (df_out['Class_cf'] == 3),
            ((df_out['Class'] == 1) & (df_out['Class_cf'] == 2)) | ((df_out['Class'] == 2) & (df_out['Class_cf'] == 1)),
            ((df_out['Class'] == 1) & (df_out['Class_cf'] == 3)) | ((df_out['Class'] == 3) & (df_out['Class_cf'] == 1)),
            ((df_out['Class'] == 2) & (df_out['Class_cf'] == 3)) | ((df_out['Class'] == 3) & (df_out['Class_cf'] == 2))]
values = ['V2V','P2P','B2B','V2P','V2B','B2P']
df_out['Conflict_Type'] = np.select(conditions,values)
'''

conditions = [(df_out['Class'] == 1) & (df_out['Class_cf'] == 1),
            (df_out['Class'] == 2) & (df_out['Class_cf'] == 2),
            (df_out['Class'] == 3) & (df_out['Class_cf'] == 3),
            (df_out['Class'] == 4) & (df_out['Class_cf'] == 4),
            (df_out['Class'] == 5) & (df_out['Class_cf'] == 5),
            ((df_out['Class'] == 1) & (df_out['Class_cf'] == 2)) | ((df_out['Class'] == 2) & (df_out['Class_cf'] == 1)),
            ((df_out['Class'] == 1) & (df_out['Class_cf'] == 3)) | ((df_out['Class'] == 3) & (df_out['Class_cf'] == 1)),
            ((df_out['Class'] == 2) & (df_out['Class_cf'] == 3)) | ((df_out['Class'] == 3) & (df_out['Class_cf'] == 2)),
            ((df_out['Class'] == 1) & (df_out['Class_cf'] == 4)) | ((df_out['Class'] == 4) & (df_out['Class_cf'] == 1)),
            ((df_out['Class'] == 2) & (df_out['Class_cf'] == 4)) | ((df_out['Class'] == 4) & (df_out['Class_cf'] == 2)),
            ((df_out['Class'] == 4) & (df_out['Class_cf'] == 3)) | ((df_out['Class'] == 3) & (df_out['Class_cf'] == 4)),
            ((df_out['Class'] == 1) & (df_out['Class_cf'] == 5)) | ((df_out['Class'] == 5) & (df_out['Class_cf'] == 1)),
            ((df_out['Class'] == 2) & (df_out['Class_cf'] == 5)) | ((df_out['Class'] == 5) & (df_out['Class_cf'] == 2)),
            ((df_out['Class'] == 3) & (df_out['Class_cf'] == 5)) | ((df_out['Class'] == 5) & (df_out['Class_cf'] == 3)),
            ((df_out['Class'] == 4) & (df_out['Class_cf'] == 5)) | ((df_out['Class'] == 5) & (df_out['Class_cf'] == 4))]
values = ['V2V','P2P','B2B','S2S','M2M','V2P','V2B','B2P','V2S','S2P','B2S','V2M','M2P','M2B','M2S']
df_out['Conflict_Type'] = numpy.select(conditions,values)


df_out['trajAngleDifference'] = df_out['directionDiff'].abs()
conditions = [(df_out['trajAngleDifference'] >180), (df_out['trajAngleDifference'] <=180)]
values = [360 - df_out['trajAngleDifference'], df_out['trajAngleDifference']]
df_out['Conflict_Angle'] = numpy.select(conditions,values)


C:\Users\Fei\AppData\Local\Temp\ipykernel_29932\2677572211.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out['DateTime'] = pd.to_datetime(df_out['DateTime'])
C:\Users\Fei\AppData\Local\Temp\ipykernel_29932\2677572211.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out['DateTime_cf'] = pd.to_datetime(df_out['DateTime_cf'])
C:\Users\Fei\AppData\Local\Temp\ipykernel_29932\2677572211.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [58]:
#df_out.to_csv(r'Z:\Virginia_SMP\Hills\Analysis\Conflicts\Hills_Conflicts.csv')
#df_out.to_csv(r'Z:\Virginia_SMP\Sierra_Manor\Analysis\Conflicts\Sierra_Manor_Conflicts.csv')
df_out.to_csv(r'Z:\NSFSCCTrajectories\KeystoneAndMcCarran\New folder\Keystone_McCarran_Conflicts_1.csv')


#arcpy.env.workspace = r"Z:\Virginia_SMP\Hills\Hills.gdb"
#arcpy.env.workspace = r"Z:\Virginia_SMP\Sierra_Manor\Sierra_Manor.gdb"
arcpy.env.workspace = r"Z:\NSFSCCTrajectories\KeystoneAndMcCarran\New folder\KeystoneAndMcCarran_conflictAnalysis.gdb"


arcpy.management.XYTableToPoint(r'Z:\NSFSCCTrajectories\KeystoneAndMcCarran\New folder\Keystone_McCarran_Conflicts.csv',"Keystone_McCarran_Conflicts_1","Longitude","Latitude")

<Result 'Z:\\NSFSCCTrajectories\\KeystoneAndMcCarran\\New folder\\KeystoneAndMcCarran_conflictAnalysis.gdb\\Keystone_McCarran_Conflicts_1'>

**Conflict_Details.py**

In [5]:
# input: extract the the data and label them roughly and then copy to a excel file.
def parse_datetime(s):
    return datetime.strptime(s, '%y-%m-%d-%H-%M-%S')
df = pd.read_excel(r"Z:\NSFSCCTrajectories\KeystoneAndMcCarran\New folder\nonV2V\Keystone_McCarran_GIS_nonV2V.xlsx")

In [6]:
df

,OBJECTID *,Shape *,Conflict_Type,timeDiff,Field1,ObjectIDs,Distance,OBJECTID_1 *,index,Timestamp,...,MoveAngle_cf,DateTime_cf,AdjSpeed_cf,MedianSpeed_cf,Acceleration_cf,directionDiff,distanceRange,trajAngleDifference,Conflict_Angle,Conflict_Movements
0,80,Point,V2S,0.20,2636,"(10524, 10541)",2.461143,46,289982,2023-10-11 13:19:11.671,...,202.087998,2023-10-11 13:19:08.500,53.394093,53.622681,6.801929,134.568976,1,134.568976,134.568976,NaN
1,68,Point,V2S,0.06,2211,"(8379, 8500)",0.919371,146,221291,2023-10-11 14:13:47.000,...,336.180226,2023-10-11 14:13:45.200,14.880926,15.730321,4.685617,41.654306,1,41.654306,41.654306,NaN
2,11,Point,V2S,0.10,178,"(623, 686)",2.601802,422,27347,2023-10-11 15:00:39.960,...,54.012833,2023-10-11 15:00:39.800,18.388100,5.505143,-2.059323,30.643533,1,30.643533,30.643533,NaN
3,15,Point,V2B,0.22,294,"(1087, 1134)",2.682090,574,32749,2023-10-11 15:01:22.060,...,115.814170,2023-10-11 15:01:18.700,29.275332,0.603173,-4.340343,85.188324,1,85.188324,85.188324,NaN
4,99,Point,V2M,0.20,3181,"(16166, 16292)",1.782131,743,473865,2023-10-11 16:58:04.370,...,197.843367,2023-10-11 16:58:01.200,60.061599,60.388021,3.387465,-120.267461,1,120.267461,120.267461,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,91,Point,V2B,0.25,2941,"(12811, 13041)",1.908853,3929,317867,2023-10-14 18:48:11.765,...,199.647450,2023-10-14 18:48:13.000,55.889577,53.661570,3.631292,-77.689740,1,77.689740,77.689740,NaN
96,51,Point,V2M,0.20,1648,"(5922, 5970)",2.614926,3990,179565,2023-10-14 21:53:42.896,...,141.063684,2023-10-14 21:53:39.700,25.263323,25.263323,<Null>,30.173013,1,30.173013,30.173013,NaN
97,69,Point,V2S,0.12,2212,"(8386, 8407)",2.365702,4023,222283,2023-10-15 08:21:59.606,...,226.729201,2023-10-15 08:21:57.200,15.300650,16.236536,<Null>,-42.079055,1,42.079055,42.079055,NaN
98,10,Point,V2P,0.14,171,"(594, 648)",1.842678,4075,22359,2023-10-15 09:32:05.996,...,212.782648,2023-10-15 09:32:03.400,11.222059,13.976663,-4.243663,111.663254,1,111.663254,111.663254,NaN


In [7]:
traj1 = '_NO'
traj2 = '_CR'
# 确保原始日期时间列的格式
# for col in ['Timestamp', 'DateTime', 'Timestamp_cf', 'DateTime_cf']:
#     if df[col].dtype != 'datetime64[ns]':
#         df[col] = pd.to_datetime(df[col])

df_out = pd.DataFrame()
df_thru = df.loc[(((df['Conflict_Type'] == 'V2P') | (df['Conflict_Type'] == 'V2S') | (df['Conflict_Type'] == 'V2B')) & (df['Class'] == 1)) |  (((df['Conflict_Type'] == 'V2V') | (df['Conflict_Type'] == 'V2M')) & (df['MedianSpeed'] >= df['MedianSpeed_cf']))]
df_thru_cf = df.loc[(((df['Conflict_Type'] == 'V2P') | (df['Conflict_Type'] == 'V2S') | (df['Conflict_Type'] == 'V2B')) & (df['Class_cf'] == 1)) |  (((df['Conflict_Type'] == 'V2V') | (df['Conflict_Type'] == 'V2M')) & (df['MedianSpeed'] < df['MedianSpeed_cf']))]

df_thru.columns = ['OID_', 'Shape','Conflict_Type', 'timeDiff','Field1','ObjectIDs','Distance','Unnamed__0','index'+ traj2,'Timestamp'+ traj2, 'ObjectID' + traj2, 'FrameIndex' + traj2,'PredInd'+traj2, 'Coord_X' + traj2, 'Coord_Y' + traj2, 'Coord_Z' + traj2, 'Speed_X' + traj2, 'Speed_Y' + traj2, 'Speed' + traj2, 'Longitude' + traj2, 'Latitude' + traj2, 'Elevation' + traj2, 'Point_Cnt' + traj2, 'Dir_X_Bbox' + traj2,
                    'Dir_Y_Bbox' + traj2, 'Height' + traj2, 'Length' + traj2, 'Width' + traj2, 'Area' + traj2, 'Dis' + traj2, 'Max_Length' + traj2, 'Class' + traj2, 'NewObjectID' + traj2, 'MoveAngle' + traj2,'DateTime' + traj2, 'AdjSpeed' + traj2, 'MedianSpeed' + traj2, 'Acceleration' + traj2, 'index'+ traj1, 'Timestamp'+ traj1,
                    'ObjectID' + traj1, 'FrameIndex' + traj1,'PredInd'+traj1, 'Coord_X' + traj1, 'Coord_Y' + traj1, 'Coord_Z' + traj1, 'Speed_X' + traj1, 'Speed_Y' + traj1, 'Speed' + traj1, 'Longitude' + traj1, 'Latitude' + traj1, 'Elevation' + traj1, 'Point_Cnt' + traj1, 'Dir_X_Bbox' + traj1,
                    'Dir_Y_Bbox' + traj1, 'Height' + traj1, 'Length' + traj1, 'Width' + traj1, 'Area' + traj1, 'Dis' + traj1, 'Max_Length' + traj1, 'Class' + traj1,'NewObjectID' + traj1, 'MoveAngle' + traj1, 'DateTime' + traj1, 'AdjSpeed' + traj1, 'MedianSpeed' + traj1, 'Acceleration' + traj1,
                     'directionDiff', 'distanceRange',  'trajAngleDifference', 'Conflict_Angle','Conflict_Movements'
]

df_thru_cf.columns = ['OID_', 'Shape','Conflict_Type','timeDiff','Field1','ObjectIDs','Distance','Unnamed__0','index'+ traj1,'Timestamp'+ traj1, 'ObjectID' + traj1, 'FrameIndex' + traj1,'PredInd'+traj1, 'Coord_X' + traj1, 'Coord_Y' + traj1, 'Coord_Z' + traj1, 'Speed_X' + traj1, 'Speed_Y' + traj1, 'Speed' + traj1, 'Longitude' + traj1, 'Latitude' + traj1, 'Elevation' + traj1, 'Point_Cnt' + traj1, 'Dir_X_Bbox' + traj1,
                    'Dir_Y_Bbox' + traj1, 'Height' + traj1, 'Length' + traj1, 'Width' + traj1, 'Area' + traj1, 'Dis' + traj1, 'Max_Length' + traj1, 'Class' + traj1, 'NewObjectID' + traj1, 'MoveAngle' + traj1,'DateTime' + traj1, 'AdjSpeed' + traj1, 'MedianSpeed' + traj1, 'Acceleration' + traj1, 'index'+ traj2, 'Timestamp'+ traj2,
                    'ObjectID' + traj2, 'FrameIndex' + traj2,'PredInd'+traj2, 'Coord_X' + traj2, 'Coord_Y' + traj2, 'Coord_Z' + traj2, 'Speed_X' + traj2, 'Speed_Y' + traj2, 'Speed' + traj2, 'Longitude' + traj2, 'Latitude' + traj2, 'Elevation' + traj2, 'Point_Cnt' + traj2, 'Dir_X_Bbox' + traj2,
                    'Dir_Y_Bbox' + traj2, 'Height' + traj2, 'Length' + traj2, 'Width' + traj2, 'Area' + traj2, 'Dis' + traj2, 'Max_Length' + traj2, 'Class' + traj2,'NewObjectID' + traj2,'MoveAngle' + traj2, 'DateTime' + traj2, 'AdjSpeed' + traj2, 'MedianSpeed' + traj2, 'Acceleration' + traj2,
                     'directionDiff', 'distanceRange', 'trajAngleDifference', 'Conflict_Angle','Conflict_Movements'
]

df_out = pd.concat([df_thru, df_thru_cf])

conditions = [df_out['FrameIndex' + traj2] < df_out['FrameIndex' + traj1], df_out['FrameIndex' + traj2] > df_out['FrameIndex' + traj1]]
values = [1,0]
df_out['First' + traj2] = np.select(conditions,values)


output_file_path = r"Z:\NSFSCCTrajectories\KeystoneAndMcCarran\New folder\nonV2V\Keystone_McCarran_GIS_CL.csv"
df_out.to_csv(output_file_path)


**Conflict_Trajectory_QC: basically generate all the trajectories related to the conlfict points**

In [12]:
df_out

,OID_,Shape,Conflict_Type,timeDiff,Field1,ObjectIDs,Distance,Unnamed__0,index_CR,Timestamp_CR,...,DateTime_NO,AdjSpeed_NO,MedianSpeed_NO,Acceleration_NO,directionDiff,distanceRange,trajAngleDifference,Conflict_Angle,Conflict_Movements,First_CR
1,68,Point,V2S,0.06,2211,"(8379, 8500)",0.919371,146,221291,2023-10-11 14:13:47.000,...,2023-10-11 14:13:45.200,14.880926,15.730321,4.685617,41.654306,1,41.654306,41.654306,NaN,0
2,11,Point,V2S,0.10,178,"(623, 686)",2.601802,422,27347,2023-10-11 15:00:39.960,...,2023-10-11 15:00:39.800,18.388100,5.505143,-2.059323,30.643533,1,30.643533,30.643533,NaN,1
3,15,Point,V2B,0.22,294,"(1087, 1134)",2.682090,574,32749,2023-10-11 15:01:22.060,...,2023-10-11 15:01:18.700,29.275332,0.603173,-4.340343,85.188324,1,85.188324,85.188324,NaN,0
5,100,Point,V2M,0.15,3183,"(16171, 16794)",1.845626,821,492210,2023-10-11 17:58:15.121,...,2023-10-11 17:58:12.400,21.329520,3.156126,-1.099341,179.444781,1,179.444781,179.444781,NaN,0
9,67,Point,V2P,0.18,2167,"(8186, 8311)",1.685698,938,239010,2023-10-11 19:17:21.410,...,2023-10-11 19:17:22.000,17.040927,5.839595,0.084019,-295.365576,1,295.365576,64.634424,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,106,Point,V2M,0.17,3408,"(25528, 25566)",0.460670,3753,548899,2023-10-14 16:19:31.645,...,2023-10-14 16:19:32.100,8.647978,11.803083,<Null>,-196.628306,1,196.628306,163.371694,NaN,1
94,94,Point,V2B,0.21,3074,"(14469, 14494)",1.838732,3886,351892,2023-10-14 17:45:00.565,...,2023-10-14 17:45:01.400,13.871667,13.458124,0.474824,39.118490,1,39.118490,39.118490,NaN,1
95,91,Point,V2B,0.25,2941,"(12811, 13041)",1.908853,3929,315036,2023-10-14 18:48:14.265,...,2023-10-14 18:48:10.500,24.080615,15.113551,1.895987,-77.689740,1,77.689740,77.689740,NaN,0
96,51,Point,V2M,0.20,1648,"(5922, 5970)",2.614926,3990,178916,2023-10-14 21:53:40.896,...,2023-10-14 21:53:41.700,20.314414,15.693199,6.574846,30.173013,1,30.173013,30.173013,NaN,1


In [13]:
df = pd.read_csv(r"Z:\NSFSCCTrajectories\KeystoneAndMcCarran\New folder\nonV2V\Keystone_McCarran_GIS_CL.csv")
df

,Unnamed: 0,OID_,Shape,Conflict_Type,timeDiff,Field1,ObjectIDs,Distance,Unnamed__0,index_CR,...,DateTime_NO,AdjSpeed_NO,MedianSpeed_NO,Acceleration_NO,directionDiff,distanceRange,trajAngleDifference,Conflict_Angle,Conflict_Movements,First_CR
0,1,68,Point,V2S,0.06,2211,"(8379, 8500)",0.919371,146,221291,...,2023-10-11 14:13:45.200,14.880926,15.730321,4.685617,41.654306,1,41.654306,41.654306,NaN,0
1,2,11,Point,V2S,0.10,178,"(623, 686)",2.601802,422,27347,...,2023-10-11 15:00:39.800,18.388100,5.505143,-2.059323,30.643533,1,30.643533,30.643533,NaN,1
2,3,15,Point,V2B,0.22,294,"(1087, 1134)",2.682090,574,32749,...,2023-10-11 15:01:18.700,29.275332,0.603173,-4.340343,85.188324,1,85.188324,85.188324,NaN,0
3,5,100,Point,V2M,0.15,3183,"(16171, 16794)",1.845626,821,492210,...,2023-10-11 17:58:12.400,21.329520,3.156126,-1.099341,179.444781,1,179.444781,179.444781,NaN,0
4,9,67,Point,V2P,0.18,2167,"(8186, 8311)",1.685698,938,239010,...,2023-10-11 19:17:22.000,17.040927,5.839595,0.084019,-295.365576,1,295.365576,64.634424,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,93,106,Point,V2M,0.17,3408,"(25528, 25566)",0.460670,3753,548899,...,2023-10-14 16:19:32.100,8.647978,11.803083,<Null>,-196.628306,1,196.628306,163.371694,NaN,1
79,94,94,Point,V2B,0.21,3074,"(14469, 14494)",1.838732,3886,351892,...,2023-10-14 17:45:01.400,13.871667,13.458124,0.474824,39.118490,1,39.118490,39.118490,NaN,1
80,95,91,Point,V2B,0.25,2941,"(12811, 13041)",1.908853,3929,315036,...,2023-10-14 18:48:10.500,24.080615,15.113551,1.895987,-77.689740,1,77.689740,77.689740,NaN,0
81,96,51,Point,V2M,0.20,1648,"(5922, 5970)",2.614926,3990,178916,...,2023-10-14 21:53:41.700,20.314414,15.693199,6.574846,30.173013,1,30.173013,30.173013,NaN,1


In [14]:
df['DateTime'] = pd.to_datetime(df['DateTime_CR'])
df['Year'] = df.DateTime.dt.year
df['Month'] = df.DateTime.dt.month
df['Day'] = df.DateTime.dt.day
df['Hour'] = df.DateTime.dt.hour
df['Minute'] = df.DateTime.dt.minute
df['Second'] = df.DateTime.dt.second

df_out = pd.DataFrame()
conditions = [(df['Minute'] < 30), (df['Minute'] >= 30)]
values = ['00','30']
df['Minute_str'] = np.select(conditions,values)



conditions = [(df['Hour'] == 0), (df['Hour'] == 1), (df['Hour'] == 2), (df['Hour'] == 3), (df['Hour'] == 4), (df['Hour'] == 5), (df['Hour'] == 6), (df['Hour'] == 7), (df['Hour'] == 8), (df['Hour'] == 9), (df['Hour'] == 10), (df['Hour'] == 11), (df['Hour'] == 12), (df['Hour'] == 13), (df['Hour'] == 14), (df['Hour'] == 15), (df['Hour'] == 16), (df['Hour'] == 17), (df['Hour'] == 18), (df['Hour'] == 19),
             (df['Hour'] == 20), (df['Hour'] == 21), (df['Hour'] == 22), (df['Hour'] == 23)]
values = ['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
df['Hour_str'] = np.select(conditions,values)


conditions = [(df['Day'] == 1), (df['Day'] == 2), (df['Day'] == 3), (df['Day'] == 4), (df['Day'] == 5), (df['Day'] == 6), (df['Day'] == 7), (df['Day'] == 8), (df['Day'] == 9), (df['Day'] == 10), (df['Day'] == 11), (df['Day'] == 12), (df['Day'] == 13), (df['Day'] == 14), (df['Day'] == 15), (df['Day'] == 16), (df['Day'] == 17), (df['Day'] == 18), (df['Day'] == 19),
             (df['Day'] == 20), (df['Day'] == 21), (df['Day'] == 22), (df['Day'] == 23), (df['Day'] == 24), (df['Day'] == 25), (df['Day'] == 26), (df['Day'] == 27), (df['Day'] == 28), (df['Day'] == 29), (df['Day'] == 30), (df['Day'] == 31)]
values = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']
df['Day_str'] = np.select(conditions,values)


conditions = [(df['Month'] ==1),(df['Month'] ==2),(df['Month'] ==3),(df['Month'] ==4),(df['Month'] ==5),(df['Month'] ==6),(df['Month'] ==7),(df['Month'] ==8),(df['Month'] ==9),(df['Month'] ==10),(df['Month'] ==11),(df['Month'] ==12)]
values = ['01','02','03','04','05','06','07','08','09','10','11','12']
df['Month_str'] = np.select(conditions,values)


conditions = [(df['Year'] == 2021),(df['Year'] == 2022),(df['Year'] == 2023)]
values = ['2021','2022','2023']
df['Year_str'] = np.select(conditions,values)


path = r"Z:\NSFSCCTrajectories\KeystoneAndMcCarran\ProcessedTrajs"
#path = r"Z:\Marketing_Sites\US50_College\Trajectories"

dir_list = os.listdir(path)
df["File_Name"] = df['Year_str'] + '-' + df['Month_str'] + '-' + df['Day_str'] + '-' + df['Hour_str'] + '-' + df['Minute_str'] + '-' + '00_cl.csv'

listed_df = []
File_Names = df.File_Name.unique()
print(File_Names)
#df_out.to_csv(r'Z:\Marketing_Sites\US50_Airport\Analysis\Conflicts\US50_College_Conflicts_GIS_cl_Traj.csv')

['2023-10-11-14-00-00_cl.csv' '2023-10-11-15-00-00_cl.csv'
 '2023-10-11-17-30-00_cl.csv' '2023-10-11-19-00-00_cl.csv'
 '2023-10-12-10-30-00_cl.csv' '2023-10-12-14-00-00_cl.csv'
 '2023-10-12-15-30-00_cl.csv' '2023-10-12-16-00-00_cl.csv'
 '2023-10-12-17-00-00_cl.csv' '2023-10-12-17-30-00_cl.csv'
 '2023-10-12-18-30-00_cl.csv' '2023-10-13-13-30-00_cl.csv'
 '2023-10-13-14-00-00_cl.csv' '2023-10-13-15-30-00_cl.csv'
 '2023-10-13-16-00-00_cl.csv' '2023-10-13-16-30-00_cl.csv'
 '2023-10-13-17-30-00_cl.csv' '2023-10-13-18-00-00_cl.csv'
 '2023-10-13-23-00-00_cl.csv' '2023-10-14-01-30-00_cl.csv'
 '2023-10-14-09-30-00_cl.csv' '2023-10-14-14-30-00_cl.csv'
 '2023-10-15-08-00-00_cl.csv' '2023-10-11-13-00-00_cl.csv'
 '2023-10-11-16-30-00_cl.csv' '2023-10-11-18-30-00_cl.csv'
 '2023-10-12-07-00-00_cl.csv' '2023-10-12-08-00-00_cl.csv'
 '2023-10-12-08-30-00_cl.csv' '2023-10-12-09-00-00_cl.csv'
 '2023-10-12-09-30-00_cl.csv' '2023-10-12-10-00-00_cl.csv'
 '2023-10-12-11-00-00_cl.csv' '2023-10-12-11-30-00_cl.cs

In [15]:
for f in File_Names:
    # print(f)
    df_file = df.loc[(df['File_Name'] == f)]
    OID_NO = df_file['ObjectID_NO'].tolist()
    OID_CR = df_file['ObjectID_CR'].tolist()
    CID = df_file['OID_'].tolist()
    #CID = df_file['Conflict_ID'].tolist()
    Conflict_Angle = df_file['Conflict_Angle'].tolist()
    timeDiff = df_file['timeDiff'].tolist()
    #timeDiff = df_file['Time_Difference'].tolist()
    Distance = df_file['Distance'].tolist()
    Conflict_Type = df_file['Conflict_Type'].tolist()
    Conflict_Movements = df_file['Conflict_Movements'].tolist()
    # print(OID_NO)
    # print(OID_CR)
    # print(len(Conflict_Movements))
    if f in dir_list:
        # print(path + '\\' + f)
        df_traj = pd.read_csv(path + '\\' + f)
        for o_NO, o_CR, c, CA, TD, dist, CT,CM in zip(OID_NO,OID_CR,CID,Conflict_Angle,timeDiff,Distance,Conflict_Type,Conflict_Movements):
        #for o_NO, o_CR, c, CA, TD, CT,CM in zip(OID_NO,OID_CR,CID,Conflict_Angle,timeDiff,Conflict_Type,Conflict_Movements):

            df_traj_temp = df_traj.loc[(df_traj['ObjectID'] == o_NO) | (df_traj['ObjectID'] == o_CR)]
            # print(c)
            # print(df_traj_temp)
            df_traj_temp["C_ID"] = c
            df_traj_temp["Conflict_Angle"] = CA
            df_traj_temp["timeDiff"] = TD
            df_traj_temp["Distance"] = dist
            df_traj_temp["Conflict_Type"] = CT
            df_traj_temp["Conflict_Movements"] = CM
            listed_df.append(df_traj_temp)

    else:
        print(f + ' is not in directory list.')

df_out = pd.concat(listed_df,axis=0)
df_out.to_csv(r'Z:\NSFSCCTrajectories\KeystoneAndMcCarran\New folder\V2V\Keystone_McCarran_nonV2V_Traj.csv')
# arcpy.env.workspace = r"Z:\RTC_Multi-Modal\Virginia_Neil\Virginia_Neil.gdb"


C:\Users\Fei\AppData\Local\Temp\ipykernel_62972\1226922313.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_traj_temp["C_ID"] = c
C:\Users\Fei\AppData\Local\Temp\ipykernel_62972\1226922313.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_traj_temp["Conflict_Angle"] = CA
C:\Users\Fei\AppData\Local\Temp\ipykernel_62972\1226922313.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [16]:
arcpy.env.workspace = r"Z:\NSFSCCTrajectories\KeystoneAndMcCarran\New folder\KeystoneAndMcCarran_conflictAnalysis.gdb"
arcpy.management.XYTableToPoint(r'Z:\NSFSCCTrajectories\KeystoneAndMcCarran\New folder\V2V\Keystone_McCarran_GIS_CL_Traj.csv',"Keystone_McCarran_nonV2V_Traj","Longitude","Latitude")

<Result 'Z:\\NSFSCCTrajectories\\KeystoneAndMcCarran\\New folder\\KeystoneAndMcCarran_conflictAnalysis.gdb\\Keystone_McCarran_nonV2V_Traj'>

In [13]:
import arcpy
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict
import os
from tqdm import tqdm
wildCard = "map2023*"


# 您提供的路径和 Shapefiles
path_output_volume = r'Z:\NSFSCCTrajectories\KeystoneAnd7thST\Analysis\Volume\Volume.csv'
path_shpFile = r'Z:\NSFSCCTrajectories\KeystoneAnd7thST\Analysis\shpFile_Volume'
geodatabase_path = r'Z:\NSFSCCTrajectories\KeystoneAnd7thST\KeystoneAnd7thST.gdb'
siteName = 'Keystone_7th'
from_zones = []
to_zones = []
for file in os.listdir(path_shpFile):
    if file.endswith('.shp') and 'VehZone' in file:
        if 'from' in file:
            from_zones.append(os.path.join(path_shpFile, file))
        elif 'to' in file:
            to_zones.append(os.path.join(path_shpFile, file))

all_data = []


# 设置工作空间
arcpy.env.workspace = geodatabase_path
arcpy.env.overwriteOutput = True

feature_classes = arcpy.ListFeatureClasses(wild_card=wildCard)
for trajectory_file in tqdm(feature_classes):
    for from_zone in from_zones:
        for to_zone in to_zones:
            site_name = siteName
            result = extract_vehicle_data(trajectory_file, from_zone, to_zone, site_name)
            all_data.append(result)

# 合并所有的 DataFrame
final_result = pd.concat(all_data, ignore_index=True)
final_result.to_csv(path_output_volume)

100%|██████████| 186/186 [31:25<00:00, 10.14s/it]


In [12]:
final_result

,ObjectID,Timestamp,Class,Direction,Site,Date,Rounded_Time
0,28,2023-10-11 13:00:04.270540953,1,EB to EB,Keystone_McCarran,2023-10-11,13:00:00
1,34,2023-10-11 13:00:04.770540953,1,EB to EB,Keystone_McCarran,2023-10-11,13:00:00
2,68,2023-10-11 13:00:09.670541048,1,EB to EB,Keystone_McCarran,2023-10-11,13:00:00
3,98,2023-10-11 13:00:13.570541143,1,EB to EB,Keystone_McCarran,2023-10-11,13:00:00
4,190,2023-10-11 13:00:29.670541048,1,EB to EB,Keystone_McCarran,2023-10-11,13:00:00
...,...,...,...,...,...,...,...
773,16250,2023-10-11 13:29:15.270540953,1,WB to WB,Keystone_McCarran,2023-10-11,13:30:00
774,16257,2023-10-11 13:29:16.170541048,1,WB to WB,Keystone_McCarran,2023-10-11,13:30:00
775,16260,2023-10-11 13:29:16.570541143,1,WB to WB,Keystone_McCarran,2023-10-11,13:30:00
776,16358,2023-10-11 13:29:26.770540953,1,WB to WB,Keystone_McCarran,2023-10-11,13:30:00


In [89]:
def Direction_Judgement(from_zone, to_zone):
    # 提取方向信息
    from_direction = from_zone.split('_')[-2]
    to_direction = to_zone.split('_')[-2]

    # 定义方向转换字典
    direction_mappings = {
        ('EB', 'EB'): 'EB_Through',
        ('EB', 'NB'): 'EB_LeftTurn',
        ('EB', 'SB'): 'EB_RightTurn',
        ('EB', 'WB'): 'EB_UTurn',
        ('SB', 'SB'): 'SB_Through',
        ('SB', 'EB'): 'SB_LeftTurn',
        ('SB', 'WB'): 'SB_RightTurn',
        ('SB', 'NB'): 'SB_UTurn',
        ('NB', 'NB'): 'NB_Through',
        ('NB', 'WB'): 'NB_LeftTurn',
        ('NB', 'EB'): 'NB_RightTurn',
        ('NB', 'SB'): 'NB_UTurn',
        ('WB', 'WB'): 'WB_Through',
        ('WB', 'SB'): 'WB_LeftTurn',
        ('WB', 'NB'): 'WB_RightTurn',
        ('WB', 'EB'): 'WB_UTurn'
    }

    # 根据方向转换字典得出结果
    return direction_mappings.get((from_direction, to_direction), "Unknown Direction")


def extract_vehicle_volume(point_layer, polygon_zone_from, polygon_zone_to, site_name):
    # 允许覆盖输出
    arcpy.env.overwriteOutput = True

    # 选择与polygon_zone1相交的点
    arcpy.MakeFeatureLayer_management(point_layer, 'tempLayer1')
    arcpy.SelectLayerByLocation_management('tempLayer1', 'WITHIN', polygon_zone_from)
    selected_zone1 = set(row[0] for row in arcpy.da.SearchCursor('tempLayer1', "ObjectID"))

    # 选择与polygon_zone2相交的点
    arcpy.MakeFeatureLayer_management(point_layer, 'tempLayer2')
    arcpy.SelectLayerByLocation_management('tempLayer2', 'WITHIN', polygon_zone_to)
    selected_zone2 = set(row[0] for row in arcpy.da.SearchCursor('tempLayer2', "ObjectID"))

    # 找到同时在两个区域中的点的ObjectID
    common_vehicles = selected_zone1.intersection(selected_zone2)

    # 从整个地图的属性表中提取相关数据
    df_all = pd.DataFrame([row for row in arcpy.da.SearchCursor(point_layer, ["ObjectID", "Timestamp", "Class"])],
                           columns=["ObjectID", "Timestamp", "Class"])

    # 根据common_vehicles筛选数据
    df_common = df_all[df_all['ObjectID'].isin(common_vehicles)]

    # 获取每个ObjectID的最小Timestamp
    df_result = df_common.groupby('ObjectID').agg({
        'Timestamp': 'min',
        'Class': 'first'
    }).reset_index()

    # 添加其他列
    direction = Direction_Judgement(polygon_zone_from, polygon_zone_to)
    df_result['Direction'] = direction
    df_result['Site'] = site_name
    df_result['Date'] = pd.to_datetime(df_result['Timestamp']).dt.date
    df_result['Rounded_15min'] = pd.to_datetime(df_result['Timestamp']).apply(simple_rounddown_time_str)

    return df_result
def round_to_nearest_30min(dt):
    # round down to 30 min
    if dt.minute < 30:
        rounded_minute = 0
    else:
        rounded_minute = 30

    return dt.replace(minute=rounded_minute, second=0, microsecond=0)
def simple_rounddown_time_str(dt):
    # Extract minutes from the datetime object
    minutes = dt.minute

    # Determine the rounded down minute value
    if minutes < 15:
        rounded_minutes = "00"
    elif minutes < 30:
        rounded_minutes = "15"
    elif minutes < 45:
        rounded_minutes = "30"
    else:
        rounded_minutes = "45"

    # Format the datetime as a string with only hours and the rounded down minutes
    return dt.strftime(f"%H:{rounded_minutes}")

def Direction_Judgement_PedZone(from_zone, to_zone):
    # 移除扩展名并提取方向信息
    from_direction = from_zone.split('_')[-1].split('.')[0]
    to_direction = to_zone.split('_')[-1].split('.')[0]

    # 组合方向信息
    return f'{from_direction} to {to_direction}'
def extract_Ped_Volume(point_layer, ped_zone_from, ped_zone_to, site_name):
    # 允许覆盖输出
    arcpy.env.overwriteOutput = True

    # 选择与 ped_zone_from 相交的点
    arcpy.MakeFeatureLayer_management(point_layer, 'tempLayer1')
    arcpy.SelectLayerByLocation_management('tempLayer1', 'WITHIN', ped_zone_from)
    selected_zone1 = set(row[0] for row in arcpy.da.SearchCursor('tempLayer1', "ObjectID"))

    # 选择与 ped_zone_to 相交的点
    arcpy.MakeFeatureLayer_management(point_layer, 'tempLayer2')
    arcpy.SelectLayerByLocation_management('tempLayer2', 'WITHIN', ped_zone_to)
    selected_zone2 = set(row[0] for row in arcpy.da.SearchCursor('tempLayer2', "ObjectID"))

    # 找到同时在两个区域中的点的ObjectID
    common_pedestrians = selected_zone1.intersection(selected_zone2)

    # 从整个地图的属性表中提取相关数据
    df_all = pd.DataFrame([row for row in arcpy.da.SearchCursor(point_layer, ["ObjectID", "Timestamp", "Class"])],
                           columns=["ObjectID", "Timestamp", "Class"])

    # 根据common_pedestrians筛选数据
    df_common = df_all[df_all['ObjectID'].isin(common_pedestrians)]

    # 获取每个ObjectID的最小Timestamp
    df_result = df_common.groupby('ObjectID').agg({
        'Timestamp': 'min',
        'Class': 'first'
    }).reset_index()

    # 获取方向
    direction = Direction_Judgement_PedZone(ped_zone_from, ped_zone_to)
    df_result['Direction'] = direction
    df_result['Site'] = site_name
    df_result['Date'] = pd.to_datetime(df_result['Timestamp']).dt.date
    df_result['Rounded_Time'] = pd.to_datetime(df_result['Timestamp']).dt.round('15min').dt.time
    return df_result
def remove_diagonal_ped_zone_to(zone_from, zones_list):
    # 定义对角线关系
    diagonals = {
        'SW': 'NE',
        'NE': 'SW',
        'SE': 'NW',
        'NW': 'SE'
    }

    # 使用正则表达式提取角落缩写
    match = re.search(r'_(SW|NE|SE|NW)\b', zone_from)
    if not match:
        raise ValueError("Cannot find corner abbreviation in zone_from string")

    corner_abbreviation = match.group(1)

    # 获取 zone_from 的对角线角落
    diagonal_corner = diagonals[corner_abbreviation]

    # 移除对角线角落
    remaining_zones = [zone for zone in zones_list if diagonal_corner not in zone]

    return remaining_zones

def get_summary_table(data, pivot_excel_path):
    # Step 1: Determine all unique 'DateTime' values
    unique_datetimes = data['DateTime'].drop_duplicates().sort_values()

    # Step 2: Determine all unique 'Direction' and 'Direction_from' values
    unique_directions = data['Direction'].drop_duplicates()
    unique_directions_from = data['Direction_from'].drop_duplicates()

    # Step 3: Create a pivot table for 'Direction'
    pivot_direction = data.pivot_table(index='DateTime',
                                       columns='Direction',
                                       aggfunc='size',
                                       fill_value=0)

    # Step 4: Create a pivot table for 'Direction_from'
    pivot_direction_from = data.pivot_table(index='DateTime',
                                            columns='Direction_from',
                                            aggfunc='size',
                                            fill_value=0)

    # Since we want both Direction and Direction_from in the same table, we can join these pivot tables
    # It's important to ensure there are no overlapping column names, hence we will prefix the column names
    pivot_direction.columns = [str(col) + '_Direction' for col in pivot_direction.columns]
    pivot_direction_from.columns = [str(col) + '_Direction_from' for col in pivot_direction_from.columns]

    # Step 5: Join the two pivot tables
    pivot_combined = pivot_direction.join(pivot_direction_from)

    # Step 6: Save the combined pivot table to a new Excel file
    pivot_combined.to_excel(pivot_excel_path)
    return pivot_combined


**inputs: shp files for detection zone, check the format! for example: for vehicle**

In [92]:
import arcpy
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict
import os
from tqdm import tqdm
import re

wildCard = "map2023101113*"

# 您提供的路径和 Shapefiles
path_output_volume_Veh = r'Z:\NSFSCCTrajectories\KeystoneAndMcCarran\New folder\Volume\Volume_Veh.csv'
path_output_volume_MM = r'Z:\NSFSCCTrajectories\KeystoneAndMcCarran\New folder\Volume\Volume_MM.csv'
path_summary_Veh = r'Z:\NSFSCCTrajectories\KeystoneAndMcCarran\New folder\Volume\Summary_Veh.xlsx'
path_shpFile = r'Z:\NSFSCCTrajectories\KeystoneAndMcCarran\New folder\ShpFiles'
geodatabase_path = r'Z:\NSFSCCTrajectories\KeystoneAndMcCarran\KeystoneAndMcCarran.gdb'
siteName = 'Keystone_McCarren'

from_zones_veh = []
to_zones_veh = []
zones_ped = []
for file in os.listdir(path_shpFile):
    if file.endswith('.shp') and 'PedZone' in file:
        zones_ped.append(os.path.join(path_shpFile, file))
for file in os.listdir(path_shpFile):
    if file.endswith('.shp') and 'VehZone' in file:
        if 'from' in file:
            from_zones_veh.append(os.path.join(path_shpFile, file))
        elif 'to' in file:
            to_zones_veh.append(os.path.join(path_shpFile, file))

all_data_veh = []
all_data_ped = []

# 设置工作空间
arcpy.env.workspace = geodatabase_path
arcpy.env.overwriteOutput = True
feature_classes = arcpy.ListFeatureClasses(wild_card=wildCard)

for trajectory_file in tqdm(feature_classes):
    for from_zone in from_zones_veh:
        for to_zone in to_zones_veh:
            site_name = siteName
            result_veh = extract_vehicle_volume(trajectory_file, from_zone, to_zone, site_name)
            all_data_veh.append(result_veh)
    for ped_zone_from in zones_ped:
        for ped_zone_to in remove_diagonal_ped_zone_to(ped_zone_from,zones_ped):
            result_ped = extract_Ped_Volume(trajectory_file, ped_zone_from, ped_zone_to, siteName)
            all_data_ped.append(result_ped)

# 合并所有的 DataFrame
if all_data_veh:
    final_result_veh = pd.concat(all_data_veh, ignore_index=True)
    final_result_veh['DateTime'] = pd.to_datetime(final_result_veh['Date'].astype(str) + ' ' + final_result_veh['Rounded_15min'].astype(str))
    # Creating a new 'Direction_from' column by splitting the 'Direction' column and taking the part before '_'
    final_result_veh['Direction_from'] = final_result_veh['Direction'].apply(lambda x: x.split('_')[0] if '_' in x else x)
    final_result_veh['map'] = final_result_veh['DateTime'].apply(round_to_nearest_30min).dt.strftime('map%Y%m%d%H%M%S')
    final_result_veh.to_csv(path_output_volume_Veh)
else:
    print("No vehicle data to write.")

if all_data_ped:
    final_result_ped = pd.concat(all_data_ped, ignore_index=True)
    final_result_ped.to_csv(path_output_volume_MM)
else:
    print("No pedestrian data to write.")

100%|██████████| 186/186 [46:31<00:00, 15.01s/it]


No pedestrian data to write.


In [93]:
final_result_veh

,ObjectID,Timestamp,Class,Direction,Site,Date,Rounded_15min,DateTime,Direction_from
0,28,2023-10-11 13:00:04.270540953,1,EB_Through,Keystone_McCarren,2023-10-11,13:00,2023-10-11 13:00:00,EB
1,34,2023-10-11 13:00:04.770540953,1,EB_Through,Keystone_McCarren,2023-10-11,13:00,2023-10-11 13:00:00,EB
2,68,2023-10-11 13:00:09.670541048,1,EB_Through,Keystone_McCarren,2023-10-11,13:00,2023-10-11 13:00:00,EB
3,98,2023-10-11 13:00:13.570541143,1,EB_Through,Keystone_McCarren,2023-10-11,13:00,2023-10-11 13:00:00,EB
4,190,2023-10-11 13:00:29.670541048,1,EB_Through,Keystone_McCarren,2023-10-11,13:00,2023-10-11 13:00:00,EB
...,...,...,...,...,...,...,...,...,...
99930,6031,2023-10-15 09:47:53.795851946,1,WB_Through,Keystone_McCarren,2023-10-15,09:45,2023-10-15 09:45:00,WB
99931,6043,2023-10-15 09:47:55.095852137,1,WB_Through,Keystone_McCarren,2023-10-15,09:45,2023-10-15 09:45:00,WB
99932,6077,2023-10-15 09:47:59.495851994,1,WB_Through,Keystone_McCarren,2023-10-15,09:45,2023-10-15 09:45:00,WB
99933,6089,2023-10-15 09:48:01.295851946,1,WB_Through,Keystone_McCarren,2023-10-15,09:45,2023-10-15 09:45:00,WB


In [94]:
get_summary_table(final_result_veh,path_summary_Veh)

,EB_LeftTurn_Direction,EB_RightTurn_Direction,EB_Through_Direction,EB_UTurn_Direction,NB_LeftTurn_Direction,NB_RightTurn_Direction,NB_Through_Direction,NB_UTurn_Direction,SB_LeftTurn_Direction,SB_RightTurn_Direction,SB_Through_Direction,SB_UTurn_Direction,WB_LeftTurn_Direction,WB_RightTurn_Direction,WB_Through_Direction,WB_UTurn_Direction,EB_Direction_from,NB_Direction_from,SB_Direction_from,WB_Direction_from
DateTime,,,,,,,,,,,,,,,,,,,,
2023-10-11 13:00:00,0,6,169,0,2,27,0,0,1,7,0,0,18,4,165,0,175,29,8,187
2023-10-11 13:15:00,0,10,172,0,0,25,0,0,3,5,0,0,20,7,137,0,182,25,8,164
2023-10-11 13:30:00,0,10,203,0,0,22,2,0,4,6,0,0,24,3,134,0,213,24,10,161
2023-10-11 13:45:00,0,10,185,0,0,19,0,0,2,6,0,0,18,3,152,0,195,19,8,173
2023-10-11 14:00:00,0,14,216,0,0,22,1,0,2,12,0,0,26,2,140,0,230,23,14,168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-15 08:45:00,0,9,80,0,0,24,0,0,0,2,1,0,12,1,142,0,89,24,3,155
2023-10-15 09:00:00,0,2,100,0,0,16,2,1,1,5,1,0,10,4,90,1,102,19,7,105
2023-10-15 09:15:00,0,5,88,0,0,14,1,0,3,1,0,0,15,1,92,0,93,15,4,108


In [99]:
import pandas as pd
import arcpy

def round_to_nearest_30min(dt):
    # round down to 30 min
    if dt.minute < 30:
        rounded_minute = 0
    else:
        rounded_minute = 30

    return dt.replace(minute=rounded_minute, second=0, microsecond=0)

def create_feature_class_from_df(df, gdb_path, output_feature_class):
    # 生成 'map' 列


    # 确保输出 feature class 不存在
    if arcpy.Exists(output_feature_class):
        arcpy.Delete_management(output_feature_class)
    arcpy.env.workspace = gdb_path
    arcpy.env.overwriteOutput = True
    # 对于每个唯一的 'map' 值
    for map_value in df['map'].unique():
        # 筛选 dataframe
        filtered_df = df[df['map'] == map_value]

        # 提取 ObjectID 并生成 IN 语句
        object_ids = filtered_df['ObjectID'].tolist()
        in_statement = ','.join(map(str, object_ids))

        # 从 geodatabase 选择并复制特征
        query = f"ObjectID IN ({in_statement})"
        selected_features = arcpy.SelectLayerByAttribute_management(map_value, selection_type="NEW_SELECTION",where_clause=query)

        # 如果是第一次循环，创建新的 feature class，否则追加特征
        if not arcpy.Exists(output_feature_class):
            arcpy.CopyFeatures_management(selected_features, output_feature_class)
        else:
            arcpy.Append_management(selected_features, output_feature_class, "NO_TEST")

    return output_feature_class

# 使用示例
df = pd.read_excel(r"Z:\NSFSCCTrajectories\KeystoneAndMcCarran\New folder\Volume\Illegal Movements.xlsx")
gdb_path = r'Z:\NSFSCCTrajectories\KeystoneAndMcCarran\KeystoneAndMcCarran.gdb'  # Geodatabase path
output_feature_class = 'Illegal_Movement'  # 输出 Feature Class 路径
create_feature_class_from_df(df, gdb_path, output_feature_class)



In [108]:
# 确保输出 feature class 不存在
if arcpy.Exists(output_feature_class):
    arcpy.Delete_management(output_feature_class)
arcpy.env.workspace = gdb_path
arcpy.env.overwriteOutput = True
# 对于每个唯一的 'map' 值
for map_value in df['map'].unique():
    # 筛选 dataframe
    filtered_df = df[df['map'] == map_value]

    # 提取 ObjectID 并生成 IN 语句
    object_ids = filtered_df['ObjectID'].tolist()
    in_statement = ','.join(map(str, object_ids))

    # 从 geodatabase 选择并复制特征
    query = f"ObjectID IN ({in_statement})"
    selected_features = arcpy.SelectLayerByAttribute_management(map_value, selection_type="NEW_SELECTION",where_clause=query)

    # 如果是第一次循环，创建新的 feature class，否则追加特征
    if not arcpy.Exists(output_feature_class):
        arcpy.CopyFeatures_management(selected_features, output_feature_class)
    else:
        arcpy.Append_management(selected_features, output_feature_class, "NO_TEST")